Trabajo con objetos relacionados
====

* Ùltima modificación: 12 de septiembre de 2022.

Preparación
----

In [1]:
from sqlalchemy import Column, ForeignKey, Integer, String
from sqlalchemy.orm import declarative_base, relationship

Base = declarative_base()


class User(Base):
    __tablename__ = "user_account"
    id = Column(Integer, primary_key=True)
    name = Column(String(30))
    fullname = Column(String)
    addresses = relationship(
        "Address", back_populates="user", cascade="all, delete-orphan"
    )

    def __repr__(self):
        return f"User(id={self.id!r}, name={self.name!r}, fullname={self.fullname!r})"


class Address(Base):
    __tablename__ = "address"
    id = Column(Integer, primary_key=True)
    email_address = Column(String, nullable=False)
    user_id = Column(Integer, ForeignKey("user_account.id"), nullable=False)
    user = relationship("User", back_populates="addresses")

    def __repr__(self):
        return f"Address(id={self.id!r}, email_address={self.email_address!r})"

In [2]:
from sqlalchemy import create_engine

engine = create_engine(
    "sqlite+pysqlite:///:memory:",
    echo=False,
    future=True,
)

In [3]:
Base.metadata.create_all(engine)

In [4]:
from sqlalchemy.orm import Session

with Session(engine) as session:

    spongebob = User(
        name="spongebob",
        fullname="Spongebob Squarepants",
        addresses=[Address(email_address="spongebob@sqlalchemy.org")],
    )

    sandy = User(
        name="sandy",
        fullname="Sandy Cheeks",
        addresses=[
            Address(email_address="sandy@sqlalchemy.org"),
            Address(email_address="sandy@squirrelpower.org"),
        ],
    )

    patrick = User(
        name="patrick",
        fullname="Patrick Star",
    )

    session.add_all(
        [
            spongebob,
            sandy,
            patrick,
        ]
    )

    session.commit()

Persistencia y carga de relaciones
---

In [5]:
u1 = User(name="pkrabs", fullname="Pearl Krabs")
u1.addresses

[]

In [6]:
a1 = Address(email_address="pearl.krabs@gmail.com")
u1.addresses.append(a1)
u1.addresses

[Address(id=None, email_address='pearl.krabs@gmail.com')]

In [7]:
a1.user

User(id=None, name='pkrabs', fullname='Pearl Krabs')

In [8]:
a2 = Address(email_address="pearl@aol.com", user=u1)
u1.addresses

[Address(id=None, email_address='pearl.krabs@gmail.com'),
 Address(id=None, email_address='pearl@aol.com')]

In [9]:
a2.user = u1

Cascada de objetos en una sesión
---

In [10]:
from sqlalchemy.orm import Session

session = Session(engine)

In [11]:
session.add(u1)
u1 in session

True

In [12]:
a1 in session

True

In [13]:
a2 in session

True

In [14]:
print(u1.id)

None


In [15]:
print(a1.user_id)

None


In [16]:
session.commit()

Carga de relaciones
----

In [17]:
u1.id

4

In [18]:
u1.addresses

[Address(id=4, email_address='pearl.krabs@gmail.com'),
 Address(id=5, email_address='pearl@aol.com')]

In [19]:
u1.addresses

[Address(id=4, email_address='pearl.krabs@gmail.com'),
 Address(id=5, email_address='pearl@aol.com')]

In [20]:
a1

Address(id=4, email_address='pearl.krabs@gmail.com')

In [21]:
a2

Address(id=5, email_address='pearl@aol.com')

Uso de relaciones en consultas
----

In [22]:
from sqlalchemy import select

print(select(Address.email_address).select_from(User).join(User.addresses))

SELECT address.email_address 
FROM user_account JOIN address ON user_account.id = address.user_id


In [23]:
print(select(Address.email_address).join_from(User, Address))

SELECT address.email_address 
FROM user_account JOIN address ON user_account.id = address.user_id


In [24]:
from sqlalchemy.orm import aliased

address_alias_1 = aliased(Address)
address_alias_2 = aliased(Address)

print(
    select(User)
    .join(User.addresses.of_type(address_alias_1))
    .where(address_alias_1.email_address == "patrick@aol.com")
    .join(User.addresses.of_type(address_alias_2))
    .where(address_alias_2.email_address == "patrick@gmail.com")
)

SELECT user_account.id, user_account.name, user_account.fullname 
FROM user_account JOIN address AS address_1 ON user_account.id = address_1.user_id JOIN address AS address_2 ON user_account.id = address_2.user_id 
WHERE address_1.email_address = :email_address_1 AND address_2.email_address = :email_address_2


In [25]:
user_alias_1 = aliased(User)
print(select(user_alias_1.name).join(user_alias_1.addresses))

SELECT user_account_1.name 
FROM user_account AS user_account_1 JOIN address ON user_account_1.id = address.user_id


ON
----

In [26]:
stmt = select(User.fullname).join(
    User.addresses.and_(Address.email_address == "pearl.krabs@gmail.com")
)
session.execute(stmt).all()

[('Pearl Krabs',)]

EXISTS
----

In [27]:
stmt = select(User.fullname).where(
    User.addresses.any(Address.email_address == "pearl.krabs@gmail.com")
)
session.execute(stmt).all()

[('Pearl Krabs',)]

In [28]:
stmt = select(User.fullname).where(~User.addresses.any())
session.execute(stmt).all()

[('Patrick Star',)]

In [29]:
stmt = select(Address.email_address).where(Address.user.has(User.name == "pkrabs"))
session.execute(stmt).all()

[('pearl.krabs@gmail.com',), ('pearl@aol.com',)]

Operadores comunes
---

In [30]:
print(select(Address).where(Address.user == u1))

SELECT address.id, address.email_address, address.user_id 
FROM address 
WHERE :param_1 = address.user_id


In [31]:
print(select(Address).where(Address.user != u1))

SELECT address.id, address.email_address, address.user_id 
FROM address 
WHERE address.user_id != :user_id_1 OR address.user_id IS NULL


In [32]:
print(select(User).where(User.addresses.contains(a1)))

SELECT user_account.id, user_account.name, user_account.fullname 
FROM user_account 
WHERE user_account.id = :param_1


In [33]:
from sqlalchemy.orm import with_parent

print(select(Address).where(with_parent(u1, User.addresses)))

SELECT address.id, address.email_address, address.user_id 
FROM address 
WHERE :param_1 = address.user_id
